In [1]:
import pickle

In [2]:
import os
from tqdm.notebook import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
import unicodedata
dataset_name = 'wikidata5m_v3'
entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))
entity_strings_set = set([unicodedata.normalize('NFKC', e) for e in entity_strings])

In [4]:
# import unicodedata
# for e in tqdm(entity_strings):
#     normalized = unicodedata.normalize('NFKC', e)
# #     if 'x86' in e:
# #         print(e)
#     if  normalized != e:
#         print(e, normalized)
# #         break


In [5]:
# entity_strings_set = set(entity_strings)

In [6]:
data = {}
splits = ['train', 'valid', 'test']
dataset_name = 'wikidata5m_v3'
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

In [7]:
# e2wdid = get_entity_wd_id_dict('data/wikidata5m/aliases.txt')
e2wdid = {}
fname = "/scratche/prod/apoorv/wd5m_aliases_entities_v3.txt"
f = open(fname)
for line in f:
    line = line.rstrip().split('\t')
    e2wdid[line[1]] = line[0]

In [8]:
e2wdid['Antarctica']

'Q51'

In [9]:
y = 'Guns of Navarone'
for k in e2wdid.keys():
    if y in k:
        print(k)

The Guns of Navarone
The Guns of Navarone - 1957 novel by Alistair MacLean
Guns of Navarone


In [10]:
filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

In [35]:
# fname = 'scores.pickle'
# fname = 'scores/scores_full_base2.pickle'
fname = 'scores/scores_wd5m_v3_500.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [36]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

In [37]:
scores_data.keys()

dict_keys(['prediction_strings', 'scores', 'target_strings', 'input_strings'])

In [38]:
id = 0
print(scores_data['prediction_strings'][id])
# print(scores_data['scores'][id])
print(scores_data['target_strings'][id])
print(scores_data['input_strings'][id])

['Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Malaysia', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada', 'Canada

In [39]:
predictions_scores_dicts[1]

defaultdict(list,
            {'Chris Hall': -10.223903,
             'Bruce MacDonald': -7.9862003,
             'Marc Tremblay': -8.600811,
             'Jim Flaherty': -9.623966,
             'Gordon Hamilton': -9.895092,
             'Charles Edwin Thompson': -11.469443,
             'John MacPherson': -8.266758,
             'Chris Stewart - Canadian ice hockey player': -9.362206,
             "Brian O'Connell": -11.241579,
             'John Carew - Canadian politician': -9.654651,
             'Gary L. Taylor': -11.8513975,
             'Jean-Claude Fournier': -9.7856245,
             'David MacDonald': -8.488233,
             'George Washington Vanderbilt': -11.625141,
             'Paul LeBlanc': -7.909206,
             'Robert MacPherson': -8.174556,
             'Jean-Luc Pépin': -10.072226,
             'Doug Jones': -9.662769,
             'George MacIntyre': -11.014691,
             'Richard Nadeau': -12.2230425,
             'John Le Mesurier': -10.019439,
             '

In [40]:
import numpy as np
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0
for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
    if 'head' in inputs:
        head_num_filter += len(filtering_entities)
    else:
        tail_num_filter += len(filtering_entities)
    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)
head_num_filter/len(predictions_filtered), tail_num_filter/len(predictions_filtered)

(1.1365344859988724, 74652.71011088141)

In [41]:
from tqdm.notebook import tqdm

count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
total_count = 0
for i in tqdm(range(len(predictions_filtered))):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    inputs = scores_data['input_strings'][i]
#     filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
#     if len(filtering_entities) > 1:
#         continue
#     else:
#         total_count += 1
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
#     if len(filtering_entities) > 1 and target in preds[:1]:
#         print(i)
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 10 and target not in preds:
        num_small_arrs += 1
        
total_count = len(predictions_filtered)
for k in k_list:
    hits_at_k = count[k]/total_count
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/total_count)
print(num_small_arrs/total_count, 'were <10 length preds array without answer')


hits@1 0.26752490133433565
hits@3 0.3177034392031573
hits@10 0.3650629580905845
mrr 0.3004872416490624
0.01531666979890998 were <10 length preds array without answer


In [33]:
total_count

10642

In [34]:
id = 4
inputs = scores_data['input_strings'][id]
predictions_unfiltered = predictions_scores_dicts[id]
predictions_unfiltered = [(k,v) for k,v in predictions_unfiltered.items()]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]
print(inputs)
print(target)
print(preds[0][0])
predictions_unfiltered.sort(key=lambda x:x[1], reverse=True)
predictions_unfiltered

|TAIL| Blank, Blank, Blank||| performer
Contrived
Blank & Jones


[('Blank & Jones', -2.1088452),
 ('Merzbow', -2.6607754),
 ('The Who', -4.3641644),
 ('Grateful Dead', -4.4439554),
 ('The Rasmus', -4.4914317),
 ('The Beach Boys', -4.5558724),
 ('The Rolling Stones', -4.570642),
 ('Dr. Dre', -4.587947),
 ('Death Cab for Cutie', -4.6841364),
 ('The Damned', -4.7946324),
 ('Acid Mothers Temple', -4.81341),
 ('Bruce Springsteen', -4.9100804),
 ('KISS', -5.0080447),
 ('Fear Factory', -5.031786),
 ('The Monkees', -5.0502644),
 ('Black Sabbath', -5.0887413),
 ('The Jesus and Mary Chain', -5.1143603),
 ('Paul Weller', -5.116393),
 ('The Smashing Pumpkins', -5.144004),
 ('Red Hot Chili Peppers', -5.2143044),
 ('The Clash', -5.234053),
 ('R.E.M.', -5.236905),
 ('Coldplay', -5.2372346),
 ('The Stranglers', -5.2707543),
 ('Die Toten Hosen', -5.288441),
 ('Dead Kennedys', -5.308741),
 ('The Prodigy', -5.381252),
 ('The Tragically Hip', -5.3837504),
 ('No Doubt', -5.4045825),
 ('King Crimson', -5.468633),
 ('The Kinks', -5.5373855),
 ('The Flaming Lips', -5.55052

In [36]:
e2wdid['archbishop']

'Q49476'

In [179]:
id = 79
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]
print(inputs)
print(target)
print(preds[0][0])
preds[:10], target, [wikidata_link_from_id(e2wdid[x[0]]) for x in preds[:10]]

predict head: dartford | located in the administrative territorial entity |
longfield and new barn
io e caterina


([('io e caterina', -4.5924783),
  ('the bigamist', -4.917049),
  ('la figlia del capitano', -5.0445466),
  ('noi siamo due evasi', -5.2959394),
  ('the great caruso', -5.5247126),
  ('la famiglia passaguai fa fortuna', -5.563276),
  ("l'amore difficile", -5.6025515),
  ('django shoots first', -5.604663),
  ('io e mia sorella', -5.6096926),
  ('la ragazza di bube', -5.6157646)],
 'longfield and new barn',
 ['https://www.wikidata.org/wiki/Q3801125',
  'https://www.wikidata.org/wiki/Q3793112',
  'https://www.wikidata.org/wiki/Q3822354',
  'https://www.wikidata.org/wiki/Q3877777',
  'https://www.wikidata.org/wiki/Q1198780',
  'https://www.wikidata.org/wiki/Q3822247',
  'https://www.wikidata.org/wiki/Q3818471',
  'https://www.wikidata.org/wiki/Q1232069',
  'https://www.wikidata.org/wiki/Q3801153',
  'https://www.wikidata.org/wiki/Q1053810'])

In [163]:
y = 'single'
wikidata_link_from_id(e2wdid[y])

'https://www.wikidata.org/wiki/Q134556'

In [482]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

32 predict tail: ali kazemaini | birthplace | tehran
34 predict tail: ashta, maharashtra | instance of | human settlement
37 predict tail: roy shaw 0 | instance of | human being
40 predict tail: t. canby jones | has surname | jones (family name)
45 predict tail: naveen kumar | instance of | human being
46 predict tail: barlow respiratory hospital | host country | united states of america
48 predict tail: camiling | office held by head of government | mayor
51 predict tail: hazel soan | instance of | human being
52 predict tail: efrain herrera | sport played | association football
53 predict tail: oluf munck | instance of | human being
54 predict tail: thomas gilchrist | instance of | human being
58 predict tail: desmoplastic fibroma | subclass of | fibroma


('count', 12)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]